# Assignment 3 ( Question 4) Know the hatred!

## Instructions
- Run this notebook on ```Google Colab(preferable)```
- Write your code and analysis in the indicated cells.
- Ensure that this notebook runs without errors when the cells are run in sequence.
- Do not attempt to change the contents of other cells. 

## Packages Used
- all the packages that are imported in the template code

## Submission
- Rename the notebook to `<roll_number>_Assignment3_Q4.ipynb`.


In [3]:
!pip install scikit-multilearn

     |████████████████████████████████| 89 kB 3.1 MB/s 


In [45]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import re
import sklearn
import torch
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import math
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import multilabel_confusion_matrix

In [5]:
!pip install transformers
import transformers
from transformers import AutoTokenizer, AutoModel

     |████████████████████████████████| 4.0 MB 4.3 MB/s 
     |████████████████████████████████| 77 kB 5.4 MB/s 
     |████████████████████████████████| 895 kB 45.1 MB/s 
     |████████████████████████████████| 596 kB 49.4 MB/s 
     |████████████████████████████████| 6.6 MB 13.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


#Dataset

In the training data, the comments are labelled as one or more of the six categories; toxic, severe toxic, obscene, threat, insult and identity hate. This is essentially a multi-label classification problem.

The dataset here is a multi-label classification dataset. To understand multi-label classification datasets, you can refer here: https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/ 

.


Here, we will be using Binary relevance to solve our multi-label classification problem



**Binary Relevance**: This is probably the simplest which treats each label as a separate single classification problems. The key assumption here though, is that there are no correlation among the various labels.

Finally, we will be summing up the accuracies obtained of all labels to obtain the final accuracy


In [6]:
# loading the dataset

train_df = pd.read_csv('train.csv',nrows=30000)
test_df = pd.read_csv('test.csv',nrows=30000)
test_labels_df=pd.read_csv('test_labels.csv',nrows=30000)
#dataset understanding
train_df.sample(10)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
19570,33b0c7689dc301bb,I am rather saddened by the small minded self ...,0,0,0,0,0,0
5961,0fe9940dd664c620,"Ways to improve Olga Alexandrova \n\nHi, I'm M...",0,0,0,0,0,0
7449,13cfff3dc28dde6e,January 2010 (UTC)\nI second Paul's thoughts. ...,0,0,0,0,0,0
6183,10817724190fcaab,"""\n After looking at a few sources it seems th...",0,0,0,0,0,0
243,009a6b3e571ee155,more info \nThanks for entering this article. ...,0,0,0,0,0,0
1630,0463507a7e208ecc,Please note that Sorenaaryamanesh is involved ...,0,0,0,0,0,0
6389,11187183be7211ad,I think the notability comes from the notabili...,0,0,0,0,0,0
16058,2a5c3937af9e9106,"Next time\nHey JK, got your note and sorry I m...",0,0,0,0,0,0
267,00ab1b314832a2d9,Could you clarify? Do you want me to start an...,0,0,0,0,0,0
676,01d4b7933d888580,"""\n— Preceding unsigned comment added by (tal...",0,0,0,0,0,0


In [7]:
test_df.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [8]:
test_labels_df=test_labels_df[(test_labels_df['toxic'] != -1) & (test_labels_df['severe_toxic'] != -1) & (test_labels_df['obscene'] != -1) & (test_labels_df['threat'] != -1) & (test_labels_df['insult'] != -1) & (test_labels_df['identity_hate'] != -1)]
test_labels_df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,0,0,0,0,0,0
7,000247e83dcc1211,0,0,0,0,0,0
11,0002f87b16116a7f,0,0,0,0,0,0
13,0003e1cccfd5a40a,0,0,0,0,0,0
14,00059ace3e3e9a53,0,0,0,0,0,0


In [9]:
final_test_df=pd.merge(test_df,test_labels_df,how="inner",on="id")
final_test_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0
1,000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0
2,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0
3,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0
4,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0


In [10]:
# let's see the total rows in train, test data and the numbers for the various categories
print('Total rows in test is {}'.format(len(test_df)))
print('Total rows in train is {}'.format(len(train_df)))
# print(train_df[cols_target].sum())

Total rows in test is 30000
Total rows in train is 30000


In [11]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [12]:
train_df['comment_text'] = train_df['comment_text'].map(lambda com : clean_text(com))
test_df['comment_text'] = final_test_df['comment_text'].map(lambda com : clean_text(com))

In [13]:
X = train_df.comment_text
test_X = final_test_df.comment_text
print(X.shape, test_X.shape)

(30000,) (12484,)


In [14]:
X

0        explanation why the edits made under my userna...
1        d aww he matches this background colour i am s...
2        hey man i am really not trying to edit war it ...
3        more i cannot make any real suggestions on imp...
4        you sir are my hero any chance you remember wh...
                               ...                        
29995    thermal and chemical are not half bad to be ho...
29996    this person has vanadalized a page about joe l...
29997    you have deemed those of us who disagree with ...
29998    i am making case for its inclusion some of the...
29999    and can we have a civil conversation about tha...
Name: comment_text, Length: 30000, dtype: object

In [15]:
test_X

0        Thank you for understanding. I think very high...
1                         :Dear god this site is horrible.
2        "::: Somebody will invariably try to add Relig...
3        " \n\n It says it right there that it IS a typ...
4        " \n\n == Before adding a new product to the l...
                               ...                        
12479    Pathetic. You're either Gothicform, or he's pa...
12480    " \n :::::(ec)I don't see how the sentence vio...
12481    :Sure it has been observed, monomers trapped i...
12482    ==  == \n\n Hi could I add this article to the...
12483    " \n : I believe if we are trying to include t...
Name: comment_text, Length: 12484, dtype: object

Now, we will be experimenting this classification problem with different classifiers. We will be using pre-trained models inorder to obtain the required embeddings. Here, we will be using BERT and Universal Sentence Encoder embeddings.


Understand how Elmo and USE(Universal Sentence Encoder) works here. You can use any of both to train your classifiers.

To understand how different sentence embeddings work, you can refer to this link: https://neptune.ai/blog/how-to-code-bert-using-pytorch-tutorial 

Please give it a look before you start working with transfer learning/ sentence embeddings like BERT, Elmo, Ulm-fit, USE.


In [16]:
#loading universal sentence encoder

import tensorflow_hub as hub
# Load pre-trained universal sentence encoder model
use_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
# elmo_embed = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

In [17]:
def get_use_embedding(sentence):
  embedding = use_embed([sentence])
  embedding = embedding.numpy()
  return embedding[0]

In [18]:
#to obtain bert embeddings ( you need not completely understand how it works)
#you can directly use the function get_bert_embedding to obtain the embedding


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')


def get_bert_embedding(sentence):
  encoded_input = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
  with torch.no_grad():
    model_output = model(**encoded_input)
  sentence_embedding = mean_pooling(model_output, encoded_input['attention_mask'])
  return sentence_embedding[0]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [19]:
def return_use_embeddings(dframe):
    use_embed_arr=[]
    for i in range(len(dframe)):
        use_embed_arr.append(get_use_embedding(X.iloc[i]))
    return np.array(use_embed_arr)

In [20]:
#Obtain X_test and X_train 
x_train_use_embed=return_use_embeddings(X)
x_test_use_embed=return_use_embeddings(test_X)

In [21]:
y_train = train_df.drop(labels = ['id','comment_text'], axis=1)
y_test = final_test_df.drop(labels = ['id','comment_text'], axis=1)

# Classifiers
Here, we use different classifiers for our classification problem. You can directly import classifiers for first two.


*   Naive Bayes classifier
*   Support Vector Machine
*   Multi-layer Perceptron(MLP)

In [22]:
#calculate the accuracies obtained, u can use sklearn metrics


In [23]:
# Naive Bayes classifier 
classifier = BinaryRelevance(GaussianNB())
classifier.fit(x_train_use_embed, y_train)
predictions = classifier.predict(x_test_use_embed)
print("Accuracy = ",accuracy_score(y_test,predictions))

Accuracy =  0.7604934315924383


In [24]:
#Support Vector Machines
classifier = BinaryRelevance(SVC())
classifier.fit(x_train_use_embed, y_train)
predictions = classifier.predict(x_test_use_embed)
print("Accuracy = ",accuracy_score(y_test,predictions))

Accuracy =  0.8338673502082666


Which classifier of the three performed better, and why do you think so ?

Ans: Support Vector Machine classifier performed better than Naive Bayes classifier as the accuracy for the later one is more, the reason for this being Support Vector Machine (SVM) is better at full-length content.

#Multi-layer Perceptron

For Multi-layer Perceptron, first you will be implementing your own articial Neural network, and then compare it with sklearn's MLP layer.


Implement your own artificial Neural Network (MLP)
Few Steps to be followed(for your reference):

*   Initialise the weights randomly
*   Decide upon the number of layers you wanna have and number of neurons associated in each layer. ( Have a clear idea of what will be the size of the weight matrices).
*   Write the code for forward and backward propogations
*   Decide upon the loss function
*   Train the model
*   Predict the labels after training the model




In [25]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def derivative_sigmoid(x):
    ones = np.ones((x.shape[0],1))
    ones = ones - x
    return x*ones


In [26]:
# you can remove returns statements if u think they are not required
# you can also add functions if needed
def initialise_weights(input_layer_sz,hidden_weights_list):
    weight_matrix_list_ans=[]
    bias_list_ans=[]
    for i in range(len(hidden_weights_list)):
        if(i==0):
            w=np.random.uniform(-1, 1, size=(input_layer_sz, hidden_weights_list[i]))
            weight_matrix_list_ans.append(w)
        else:
            w=np.random.uniform(-1, 1, size=(hidden_weights_list[i-1], hidden_weights_list[i]))
            weight_matrix_list_ans.append(w)
    for i in range(len(hidden_weights_list)):
        if(i==0):
            continue
        else:
            b=np.random.uniform(-1, 1, size=(1,hidden_weights_list[i]))
            bias_list_ans.append(b)
    return weight_matrix_list_ans,bias_list_ans
  

def loss_function(y_pred,y_actual):
    corr_prob=0
    if(y_actual==0 and y_pred > 0.50):
        corr_prob=1-y_pred
    elif(y_actual==1 and y_pred < 0.50):
        corr_prob=1-y_pred
    else:
        corr_prob=y_pred
    return -1*np.log(corr_prob)

def derivative_loss_function(p,y):
    return ((p-y)/(p*(1-p)))
    
def forward_layer(input_matrix,weight_matrix_list,bias_list):
    hidden_layers_ans=[]
    for i in range(len(weight_matrix_list)):
        curr_weight_matrix=weight_matrix_list[i]
        if(i==0):
            ans_matrix=np.dot(input_matrix,curr_weight_matrix)
            ans_matrix=sigmoid(ans_matrix)
            hidden_layers_ans.append(ans_matrix)
        else:
            bias_matrix=bias_list[i-1]
            ip_matrix=hidden_layers_ans[-1]
            ans_matrix=np.dot(ip_matrix,curr_weight_matrix)
            ans_matrix=ans_matrix+bias_matrix
            ans_matrix=sigmoid(ans_matrix)
            hidden_layers_ans.append(ans_matrix)
    return hidden_layers_ans
    
def backward_propogation(input_matrix,loss_der,weight_matrix_list,layers_input):
    updated_weights=[]
    updated_biases=[]
    repeated_value=loss_der*derivative_sigmoid(layers_input[-1])
    list_length=len(layers_input)
    for i in range(len(weight_matrix_list)-1):
        new_weight= layers_input[list_length-i-2].T * repeated_value.T
        new_bias=repeated_value.T
        updated_weights.append(new_weight)
        updated_biases.append(new_bias)  
        intermediate_value=weight_matrix_list[list_length-i-1]*derivative_sigmoid(layers_input[list_length-i-2].T)
        repeated_value = np.matmul(intermediate_value,repeated_value)
    
    new_weight=input_matrix.T*repeated_value.T
    updated_weights.append(new_weight)
    updated_weights.reverse()
    updated_biases.reverse()
    return updated_weights,updated_biases
    


In [27]:
def update_weights(original_weight,bias,updated_weight,updated_bias):
    for i in range(len(original_weight)):
        original_weight[i]=original_weight[i]-0.1*updated_weight[i]
    for i in range(len(bias)):
        bias[i]=bias[i]-0.1*updated_bias[i]
    return original_weight,bias
    

In [28]:
x_train_use_embed.shape

(30000, 512)

In [35]:
def print_results(y_pred,y_true,col_name):
    print("RESULTS FOR",col_name)
    print("Confusion Matrix = ",confusion_matrix(y_true, y_pred))
    acc=accuracy_score(y_true,y_pred)
    print("Accuracy = ",acc)
    print("F1-score = ",f1_score(y_true, y_pred))
    print()
    return acc

In [39]:

def train(x_train_embed,hidden_layers_list,col_name):
    input_layer_size=x_train_embed.shape[1]
    my_weights,my_biases=initialise_weights(input_layer_size,hidden_layers_list)
    for j in range(10):
        for row in range(x_train_embed.shape[0]):
            temp  = x_train_embed[row].reshape(1,512)
            my_layer=forward_layer(temp,my_weights,my_biases)
            y_pred=my_layer[-1]
            y_true = y_train[col_name][row]
            loss_der = derivative_loss_function(y_pred, y_true)[0]
            updated_weights, updated_bias = backward_propogation(temp,loss_der,my_weights,my_layer)
            my_weights,my_biases = update_weights(my_weights, my_biases, updated_weights, updated_bias)
        # print("Epoch", j+1)
        
    return my_weights,my_biases

In [40]:
hidden_layers_arr=[128,16,1]
final_weights_1,final_bias_1=train(x_train_use_embed,hidden_layers_arr,'toxic')
final_weights_2,final_bias_2=train(x_train_use_embed,hidden_layers_arr,'severe_toxic')
final_weights_3,final_bias_3=train(x_train_use_embed,hidden_layers_arr,'obscene')
final_weights_4,final_bias_4=train(x_train_use_embed,hidden_layers_arr,'threat')
final_weights_5,final_bias_5=train(x_train_use_embed,hidden_layers_arr,'insult')
final_weights_6,final_bias_6=train(x_train_use_embed,hidden_layers_arr,'identity_hate')


In [41]:
#report accuracies
#here, you need to predict for each of the label as we have trained classifiers in such a way

def predict(fw,fb,col_name):
    y_pred=[]
    for row in range(x_test_use_embed.shape[0]):
        temp  = x_test_use_embed[row].reshape(1,512)
        final_layer=forward_layer(temp,fw,fb)
        if(final_layer[-1][0][0]>0.5):
            y_pred.append(1)
        else:
            y_pred.append(0)
    y_true = list(y_test[col_name])
    acc=print_results(y_pred,y_true,col_name)
    return acc


In [42]:
#report accuracies on the test dataset 
acc1=predict(final_weights_1,final_bias_1,'toxic')
acc2=predict(final_weights_2,final_bias_2,'severe_toxic')
acc3=predict(final_weights_3,final_bias_3,'obscene')
acc4=predict(final_weights_4,final_bias_4,'threat')
acc5=predict(final_weights_5,final_bias_5,'insult')
acc6=predict(final_weights_6,final_bias_6,'identity_hate')
averageAcc=(acc1+acc2+acc3+acc4+acc5+acc6)/6

RESULTS FOR toxic
Confusion Matrix =  [[10318   969]
 [ 1111    86]]
Accuracy =  0.8333867350208266
F1-score =  0.07637655417406748

RESULTS FOR severe_toxic
Confusion Matrix =  [[12387    27]
 [   70     0]]
Accuracy =  0.9922300544697212
F1-score =  0.0

RESULTS FOR obscene
Confusion Matrix =  [[11243   521]
 [  689    31]]
Accuracy =  0.9030759371996155
F1-score =  0.04874213836477988

RESULTS FOR threat
Confusion Matrix =  [[12425    13]
 [   46     0]]
Accuracy =  0.9952739506568408
F1-score =  0.0

RESULTS FOR insult
Confusion Matrix =  [[11252   583]
 [  620    29]]
Accuracy =  0.9036366549182954
F1-score =  0.045995241871530534

RESULTS FOR identity_hate
Confusion Matrix =  [[12276    59]
 [  149     0]]
Accuracy =  0.9833386735020827
F1-score =  0.0



Now, compare your implemented MLP with the sklearn MLP layer.

In [46]:
classifier = BinaryRelevance(MLPClassifier(hidden_layer_sizes=(128,16)))
classifier.fit(x_train_use_embed, y_train)
predictions = classifier.predict(x_test_use_embed)
acc_builtin=accuracy_score(y_test,predictions)
#report accuracies both on train dataset and test dataset
print("Accuracy on test = ",acc_builtin)
m_CM=multilabel_confusion_matrix(predictions, y_test)
predictions = classifier.predict(x_train_use_embed)
print("Accuracy on train = ",accuracy_score(y_train,predictions))

Accuracy on test =  0.81247997436719
Accuracy on train =  0.9991


Which MLP performed better? 
Understand and Analyse.

Although the accuracy of the MLP built from scratch is greater than Sklearn MLPClassifier and also the confusion matrixes for the both are approximately same. So, in terms of accuracy the one built from scratch performs better.

In [47]:
print("Accuracy On BuiltIn CLassifier= ",acc_builtin)
print("Accuracy On MY CLassifier= ",averageAcc)

Accuracy On BuiltIn CLassifier=  0.81247997436719
Accuracy On MY CLassifier=  0.9351570009612303


In [48]:
print("Confusion Matrix for BuiltIn CLassifier= ",m_CM)

Confusion Matrix for BuiltIn CLassifier=  [[[10204  1094]
  [ 1083   103]]

 [[12287    70]
  [  127     0]]

 [[11154   684]
  [  610    36]]

 [[12401    46]
  [   37     0]]

 [[11256   624]
  [  579    25]]

 [[12227   149]
  [  108     0]]]
